In [1]:
import pandas as pd
import cv2
import numpy as np
from skimage.morphology import white_tophat
from skimage.morphology import disk
from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.transform import warp, AffineTransform
import gcsfs
from utils import *
from fstack_cu import fstack_cu_images

In [2]:
# script settings
channels = [405,488,561,638]
colors = {'0':[255,255,255],'1':[255,200,0],'2':[30,200,30],'3':[0,0,255]} # BRG

# FOV setting
imax = 5 # row of the scan, ranges 0-5
jmax = 5 # column of the scan, ranges 0-5
kmax = 9 # z stack layer, ranges 0-9

# crop settings
crop_start = 1000
crop_end = 2500
a = crop_end - crop_start

# gcs setting
gcs_project = 'soe-octopi'
gcs_token = 'codex-20220324-keys.json'
gcs_settings = {}
gcs_settings['gcs_project'] = gcs_project
gcs_settings['gcs_token'] = gcs_token
fs = gcsfs.GCSFileSystem(project=gcs_project,token=gcs_token)

# dataset ID
bucket_source = 'gs://octopi-codex-data'
bucket_destination = 'gs://octopi-codex-data-processing'
experiment_id = '20220601_20x_75mm'

with fs.open( bucket_source + '/' + experiment_id + '/' + 'index.csv', 'r' ) as f:
    database_df = pd.read_csv(f)

In [3]:
# Pick which channels to read
n = database_df.shape[0] # n is the number of cycles
import time
t0 = time.time()
I_stack = np.zeros((kmax+1,a,a))
for i in range(imax):
    for j in range(jmax):
        for channel in channels:
            for index, row in database_df.iterrows():
                for k in range(kmax):
                    filename = row['Acquisition_ID'] + '/0/' + str(i) + '_' + str(j) + '_' + str(k) + '_Fluorescence_' + str(channel) + '_nm_Ex.bmp'
                    I = imread_gcsfs(fs,bucket_source + '/' + experiment_id + '/'+ filename)
                    I = I[crop_start:crop_end,crop_start:crop_end]
                    I = I - np.amin(I)      
                    
                    I = I.astype('float')
                    # I = 255*I/np.amax(I)
                    I_stack[k,:,:] = I

                # We now have a full stack of images
                img = fstack_cu_images(I_stack, list(range(kmax)))
                cv2.imwrite("../images/all/"+row['Acquisition_ID'] + str(i) + "_" + str(j) + ".png",img)
                # selem = disk(30)
                # img = white_tophat(img,selem)
                # cv2.imwrite("../images/cloudimages_full_5/"+row['Acquisition_ID'] + str(i) + "_" + str(j) + "rmbg.png",img)

t1 = time.time()
with open('../images/all/time.txt', 'w') as f:
    f.write(str(t1-t0))

FMeasure: 0.47769665718078613
SMeasure
gauss3P: 0.22930908203125
err: 0.20845699310302734
filter: 0.12352800369262695
fusion: 0.04710078239440918
total time: 1.0860915184020996
FMeasure: 0.222381591796875
SMeasure
gauss3P: 0.21709203720092773
err: 0.1611011028289795
filter: 0.11355781555175781
fusion: 0.05142617225646973
total time: 0.7655587196350098
FMeasure: 0.22768807411193848
SMeasure
gauss3P: 0.21732854843139648
err: 0.16118240356445312
filter: 0.11812543869018555
fusion: 0.04611682891845703
total time: 0.7704412937164307
FMeasure: 0.22726917266845703
SMeasure
gauss3P: 0.203965425491333
err: 0.1566941738128662
filter: 0.11317753791809082
fusion: 0.050988197326660156
total time: 0.7520945072174072
FMeasure: 0.23862886428833008
SMeasure
gauss3P: 0.22546720504760742
err: 0.16459393501281738
filter: 0.11812829971313477
fusion: 0.046512603759765625
total time: 0.7933309078216553
FMeasure: 0.23868083953857422
SMeasure
gauss3P: 0.21245908737182617
err: 0.16896867752075195
filter: 0.1141

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
im1 = np.array(cv2.imread("cloudimages/20x_cycle12_6_well_plate_6x6_z_stack_2022-06-03_18-07-1.0111632_2rmbg2.png")[:,:,0], dtype='f')
im2 = np.array(cv2.imread("cloudimages/20x_cycle12_6_well_plate_6x6_z_stack_2022-06-03_18-07-1.0111632_2.png")[:,:,0], dtype = 'f')
dif = im2-im1
%matplotlib inline
plt.imshow(np.abs(dif), interpolation='nearest')
plt.colorbar()
plt.show()
print(100*np.sum(np.abs(dif) >= 1)/dif.size)
print((np.min(dif), np.max(dif)))
print(np.mean(np.abs(dif[np.abs(dif)>=1]/im1[np.abs(dif)>=1]) * 100))